In [3]:
import pandas as pd
import sklearn as sk

In [4]:
# load dataset/train_test/train.csv
train = pd.read_csv('../dataset/train_test/train.csv')

# load dataset/train_test/test.csv
test = pd.read_csv('../dataset/train_test/test.csv')

train

,Title,Political Lean,Score,Id,Subreddit,URL,Num of Comments,Text,Date Created
0,"No matter who someone is, how they look like, ...",Liberal,1,t5fybt,socialism,https://v.redd.it/ng5fyl7hp2l81,0,NaN,1.646272e+09
1,Biden speech draws 38.2 million U.S. TV viewers,Liberal,6,t5fqdn,democrats,https://www.reuters.com/world/us/biden-speech-...,1,NaN,1.646271e+09
2,State of the union,Liberal,1,t5fj9a,DemocraticSocialism,https://www.reddit.com/r/DemocraticSocialism/c...,1,Who watched the state of the union last night ...,1.646270e+09
3,We Should Just Give Poor People Money,Liberal,7,t5f7n9,SocialDemocracy,https://youtu.be/a80kRjpubG0,3,NaN,1.646270e+09
4,Do it for the Dew,Liberal,6,t5es2c,democrats,https://i.redd.it/drmunn90f2l81.jpg,1,NaN,1.646268e+09
...,...,...,...,...,...,...,...,...,...
10278,There's nothing wrong with minimizing tax,Conservative,95,r4ktp4,Capitalism,https://www.youtube.com/watch?v=DBg7DnQjjcY&ab...,47,NaN,1.638152e+09
10279,The MYTH of Trickle Down Economics,Conservative,10,r4i1w8,Capitalism,https://youtu.be/kqsT1aYFzk0,31,NaN,1.638144e+09
10280,"""In dealing with our economy, more is in quest...",Conservative,0,r49dhy,Capitalism,https://www.reddit.com/r/Capitalism/comments/r...,0,"""In dealing with our economy, more is in quest...",1.638120e+09
10281,If you lived as an average person in 1900 woul...,Conservative,0,r45qlx,Capitalism,https://www.reddit.com/r/Capitalism/comments/r...,20,"First, I do believe that capitalism, with some...",1.638110e+09


In [6]:
X_train, y_train = train['Title'], train["Political Lean"]
X_test, y_test = test['Title'], test["Political Lean"]

import tensorflow as tf

2024-03-13 14:43:42.363717: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-13 14:43:42.446743: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [59]:
import tensorflow as tf
from sklearn.model_selection import train_test_split

# Assuming X are the inputs (titles) and Y are the labels (political orientation)
#X = ["Trump is going to win", "Biden is going to win", "Trump is a conservative", "Biden is liberal"]
#Y = ["Conservative", "Liberal", "Conservative", "Liberal"]

X, Y = train['Title'], train["Political Lean"]
X_test, y_test = test['Title'], test["Political Lean"]

# Convert labels to numeric values
unique_labels = set(Y)  # Get the unique labels
label_tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=None, filters='', lower=False, split=' ', char_level=False, oov_token=None, document_count=0)
label_tokenizer.fit_on_texts(unique_labels)
label_seq = label_tokenizer.texts_to_sequences(Y)

# Subtract 1 from each element in the nested lists
label_seq = [[token - 1 for token in label] for label in label_seq]

Y_seq = tf.keras.utils.to_categorical(label_seq, num_classes=len(unique_labels))

# Split data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y_seq, test_size=0.2, random_state=42)

In [60]:

print(len(X_train), len(X_test), len(Y_train), len(Y_test))
# Tokenize the words
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Padding sequences to ensure uniform input size
X_train_padded = tf.keras.preprocessing.sequence.pad_sequences(X_train_seq, padding='post')
X_test_padded = tf.keras.preprocessing.sequence.pad_sequences(X_test_seq, padding='post')

8226 2057 8226 2057


In [65]:
# # Define a simple RNN model
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=16),
    tf.keras.layers.SimpleRNN(units=128),
    tf.keras.layers.Dense(2, activation='softmax')
])

# load model
#model = tf.keras.models.load_model('20240313-151915_rnn.h5')

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print(X_train_padded.shape, Y_train.shape)
print(X_train_padded)

# Train the model
model.fit(X_train_padded, Y_train, epochs=150, validation_data=(X_test_padded, Y_test))

# Save model with current date and parameters as filename
import datetime
filename = datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + '_rnn.h5'
model.save(filename)


(8226, 60) (8226, 2)
[[   39    17   281 ...     0     0     0]
 [ 6971     0     0 ...     0     0     0]
 [   59   242   782 ...     0     0     0]
 ...
 [ 4139    70   186 ...     0     0     0]
 [14284     7     1 ...     0     0     0]
 [ 2244     5  2754 ...     0     0     0]]
Epoch 1/150
258/258 ━━━━━━━━━━━━━━━━━━━━ 9s 23ms/step - accuracy: 0.7419 - loss: 0.5576 - val_accuracy: 0.3656 - val_loss: 0.9658
Epoch 2/150
258/258 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - accuracy: 0.7424 - loss: 0.5517 - val_accuracy: 0.3646 - val_loss: 1.0000
Epoch 3/150
258/258 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - accuracy: 0.7494 - loss: 0.5551 - val_accuracy: 0.3704 - val_loss: 0.8935
Epoch 4/150
258/258 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - accuracy: 0.7502 - loss: 0.5457 - val_accuracy: 0.3709 - val_loss: 0.9229
Epoch 5/150
258/258 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - accuracy: 0.7454 - loss: 0.5527 - val_accuracy: 0.3714 - val_loss: 0.9598
Epoch 6/150
258/258 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - accuracy:

KeyboardInterrupt: 